In [ ]:
import pandas as pd
import pandas_datareader.data as pdr
import datetime as dt
import matplotlib.pyplot as plt

tickers = ['AAPL', 'AMZN', 'TSLA', 'NFLX', 'BHC', 'ABBV', 'FB']

           #, 'AMD', 'SPY', 'GOOG', 'T', 'BAC', 'AVGO',
           #'COG', 'CERN', 'CSCO', 'KO', 'FANG', 'ETFC', 'EA', 'FB', 'HOG', 'HD', 'INTU', 'IVZ', 'MA', 'MCD', 'MU', 
           #'NDAQ', 'NKE', 'ORCL', 'PEP', 'QCOM', 'SNAP', 'TWTR', 'TRIP', 'UA', 'VZ', 'V', 'XRX']

stock_cp = pd.DataFrame()
attempts = 0
successful_tickers = []

while len(tickers) != 0 and attempts <= 5:
    tickers = [j for j in tickers if j not in successful_tickers]
    for i in range(len(tickers)):
        try:
            temp = pdr.get_data_yahoo(tickers[i], dt.date.today() - dt.timedelta(3650), dt.date.today())
            temp.dropna(inplace = True)
            stock_cp[tickers[i]] = temp['Adj Close']
            successful_tickers.append(tickers[i])
        except:
            print(tickers[i], 'failed to extract data.... retrying.')
            continue
    attempts += 1
    stock_cp.fillna(axis = 0, method = 'backfill', inplace = True)
    #axis = 0 means operation is performed on the columns an axis = 1 means operation is performed on the rows
    #inplace takes a bool: if True then the initial data values are changed if False then only for showing no change in real data.
    stock_cp.dropna(axis = 0, inplace = False)
    
    # statistics of our data: mean, median, standard deviation(std function).
    # default axis is 0
    stock_cp.mean()
    stock_cp.median()
    stock_cp.std()
    
    # IMP daily return
    daily_return = stock_cp.pct_change()
    daily_return.mean()
    daily_return.std()
    
    #rolling averages are the SMA's nad EMA's
    #rolling has a min_periods for less nan values but not really accurate
    daily_return.rolling(window = 20).mean()
    daily_return.rolling(window = 20).std()
    daily_return.ewm(span = 20)
    
    #to plot the whole dataframe
    stock_cp.plot()
    #standardisation of the data z-score
    stock_cp_standardized = (stock_cp - stock_cp_mean())/stock_cp.std()
    stock_cp_standardized.plot()
    #subplots for each individual stocks if we have a dataframe we dont need pyplot
    stock_cp.plot(subplots = True, figsize = (10, 30), title = 'stock proce history')
    #now pyplot from matplotlib when we dont have pandas dataframe we may have numpy 
    fig, ax = plt.subplots()
    plt.style.use('ggplot')
    ax.set(title = 'Daily Return On Tech Stocks', xlabel = 'Tech Stocks', ylabel = 'Daily Averge Return')
    plt.bar(daily_return.columns, daily_return.mean())
    
    #plotting the data
    fig, ax = plt.subplots()
    plt.style.use('ggplot')
    ax.set(title = 'Daily Return On Tech Stocks', xlabel = 'Tech Stocks', ylabel = 'Daily Averge Return')
    plt.bar(stock_cp.pct_change().columns, stock_cp.pct_change().mean())